In [100]:
import os
import pandas as pd
safety_analysis_dir = "../../data/analyzed/catHarmQA/"

# import warnings
# warnings.filterwarnings("ignore")
# pd.set_option("display.max_rows", None)

response_safety_analysis_df = pd.read_csv(os.path.join(safety_analysis_dir, "llamaguard_safety_label_analysis.csv"))
question_safety_analysis_df = pd.read_csv(os.path.join(safety_analysis_dir, "llamaguard_safety_label_analysis_questions.csv"))

In [101]:
# Check if the number of rows in the response_safety_analysis_df is 4 times the number of rows in the question_safety_analysis_df, cause there are 4 models in the response_safety_analysis_df
response_safety_analysis_df.__len__() == question_safety_analysis_df.__len__() * 4

True

In [102]:
question_safety_analysis_df = question_safety_analysis_df.rename(
    columns={
        "safe_per": "safe_per_q",
        "unsafe_per": "unsafe_per_q",
        "safe_count": "safe_count_q",
        "unsafe_count": "unsafe_count_q",
    }
)

In [103]:
response_safety_analysis_df = response_safety_analysis_df.rename(
    columns={
        "safe_per": "safe_per_r",
        "unsafe_per": "unsafe_per_r",
        "safe_count": "safe_count_r",
        "unsafe_count": "unsafe_count_r",
    }
)

response_safety_analysis_df["experiment_r"] = [
    (
        "_".join(exp.split("_")[:-1])
        if len(exp.split("_")) != 2
        else exp.split("_")[0] + "_naive"
    )
    for exp in response_safety_analysis_df.experiment
]

In [104]:
print(f"response_safety_analysis_df.columns: {response_safety_analysis_df.columns}")
print(f"question_safety_analysis_df.columns: {question_safety_analysis_df.columns}")

response_safety_analysis_df.columns: Index(['experiment', 'safe_per_r', 'unsafe_per_r', 'safe_count_r',
       'unsafe_count_r', 'model', 'perturbation_level', 'perturbation_type',
       'perturbation_count', 'experiment_r'],
      dtype='object')
question_safety_analysis_df.columns: Index(['experiment', 'safe_per_q', 'unsafe_per_q', 'safe_count_q',
       'unsafe_count_q', 'perturbation_level', 'perturbation_type',
       'perturbation_count'],
      dtype='object')


In [105]:
combined_df = pd.merge(
    response_safety_analysis_df,
    question_safety_analysis_df,
    how="inner",
    left_on=[
        "experiment_r",
        "perturbation_level",
        "perturbation_type",
        "perturbation_count",
    ],
    right_on=[
        "experiment",
        "perturbation_level",
        "perturbation_type",
        "perturbation_count",
    ],
    suffixes=("_response", "_question"),
)

# Drop duplicate 'experiment' columns if needed
combined_df = combined_df.drop(columns=["experiment_r"])
combined_df = combined_df.rename(columns={"experiment_response": "experiment", "experiment_question": "experiment_q"})

In [106]:
desired_order = [ "experiment", "safe_per_q", "safe_per_r", "unsafe_per_q", "unsafe_per_r", "safe_count_q", "safe_count_r", "unsafe_count_q", "unsafe_count_r", "perturbation_level", "perturbation_type", "perturbation_count", "experiment_q"]

In [107]:
combined_df = combined_df[desired_order]

In [108]:
combined_df

,experiment,safe_per_q,safe_per_r,unsafe_per_q,unsafe_per_r,safe_count_q,safe_count_r,unsafe_count_q,unsafe_count_r,perturbation_level,perturbation_type,perturbation_count,experiment_q
0,Question_word_random_substitute_cwe_n5_llama2,37.64,60.55,62.36,39.45,207,333,343,217,word,random_substitute_cwe,5.0,Question_word_random_substitute_cwe_n5
1,Question_word_random_substitute_cwe_n5_mistral,37.64,55.09,62.36,44.91,207,303,343,247,word,random_substitute_cwe,5.0,Question_word_random_substitute_cwe_n5
2,Question_word_random_substitute_cwe_n4_llama2,30.18,54.91,69.82,45.09,166,302,384,248,word,random_substitute_cwe,4.0,Question_word_random_substitute_cwe_n4
3,Question_word_random_substitute_cwe_n5_llama3,37.64,49.64,62.36,50.36,207,273,343,277,word,random_substitute_cwe,5.0,Question_word_random_substitute_cwe_n5
4,Question_word_random_substitute_cwe_n3_llama2,23.09,49.27,76.91,50.73,127,271,423,279,word,random_substitute_cwe,3.0,Question_word_random_substitute_cwe_n3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,Question_char_ocr_n5_llama3,5.82,13.82,94.18,86.18,32,76,518,474,char,ocr,5.0,Question_char_ocr_n5
240,Question_word_random_insert_emb_n1_llama3,7.09,13.45,92.91,86.55,39,74,511,476,word,random_insert_emb,1.0,Question_word_random_insert_emb_n1
241,Question_word_spelling_n2_llama3,5.45,13.45,94.55,86.55,30,74,520,476,word,spelling,2.0,Question_word_spelling_n2
242,Question_word_random_insert_emb_n5_llama3,10.00,12.73,90.00,87.27,55,70,495,480,word,random_insert_emb,5.0,Question_word_random_insert_emb_n5


In [109]:
combined_df.to_csv(os.path.join(safety_analysis_dir, "llamaguard_safety_label_analysis_combined.csv"), index=False)